In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler

import warnings
warnings.filterwarnings('ignore')

Available NASA POWER Parameters:
features = {
    'T2M': 'Temperature at 2 Meters (°C)',
    'T2M_MIN': 'Minimum Temperature',
    'T2M_MAX': 'Maximum Temperature',
    'PRECTOTCORR': 'Precipitation Corrected (mm/day)',
    'WS10M': 'Wind Speed at 10m',
    'RH2M': 'Relative Humidity at 2m',
    'PS': 'Surface Pressure',
    'ALLSKY_SFC_SW_DWN': 'Solar Radiation'
}

Target Variables (Multi-Output Classification):
targets = {
    'very_hot_risk': 'Binary (1 if T2M_MAX > 90th percentile)',
    'very_cold_risk': 'Binary (1 if T2M_MIN < 10th percentile)',
    'very_windy_risk': 'Binary (1 if WS10M > 90th percentile OR >40mph)',
    'very_wet_risk': 'Binary (1 if PRECTOTCORR > 90th percentile OR >76mm)',
    'uncomfortable_risk': 'Binary (1 if Heat Index >103°F OR Wind Chill <-20°F)'
}

In [12]:
#Historical Probability Model
# Simpler, interpretable approach using statistical methods
from sklearn.model_selection import train_test_split

def calculate_historical_probabilities(data, date, window_days=7):
    """
    For a given date, calculate probabilities based on
    historical frequency within ±window_days
    """
    target_day_of_year = date.timetuple().tm_yday

    # Filter data to similar dates across all years
    mask = (data.index.dayofyear >= target_day_of_year - window_days) & \
           (data.index.dayofyear <= target_day_of_year + window_days)
    historical_data = data[mask]

    # Calculate percentile thresholds
    thresholds = {
        'very_hot': historical_data['T2M_MAX'].quantile(0.90),
        'very_cold': historical_data['T2M_MIN'].quantile(0.10),
        'very_windy': historical_data['WS10M'].quantile(0.90),
        'very_wet': historical_data['PRECTOTCORR'].quantile(0.90)
    }

    # Calculate probabilities
    probabilities = {
        'very_hot': (historical_data['T2M_MAX'] > thresholds['very_hot']).mean(),
        'very_cold': (historical_data['T2M_MIN'] < thresholds['very_cold']).mean(),
        'very_windy': (historical_data['WS10M'] > thresholds['very_windy']).mean(),
        'very_wet': (historical_data['PRECTOTCORR'] > thresholds['very_wet']).mean()
    }

    return probabilities, thresholds

ML Classification Model

In [4]:
#ML Classification Model

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.multioutput import MultiOutputClassifier
import xgboost as xgb

# Feature engineering
def create_features(df):
    df['day_of_year'] = df.index.dayofyear
    df['month'] = df.index.month
    df['year'] = df.index.year

    # Rolling statistics (7-day window)
    df['temp_7day_mean'] = df['T2M'].rolling(7).mean()
    df['temp_7day_std'] = df['T2M'].rolling(7).std()
    df['precip_7day_sum'] = df['PRECTOTCORR'].rolling(7).sum()

    # Lag features
    df['temp_lag1'] = df['T2M'].shift(1)
    df['temp_lag7'] = df['T2M'].shift(7)
    df['precip_lag1'] = df['PRECTOTCORR'].shift(1)

    # Seasonal indicators
    df['is_monsoon'] = ((df['month'] >= 6) & (df['month'] <= 9)).astype(int)
    df['is_winter'] = ((df['month'] >= 11) | (df['month'] <= 2)).astype(int)

    return df.dropna()

# Multi-output classification
model = MultiOutputClassifier(
    xgb.XGBClassifier(
        n_estimators=100,
        max_depth=5,
        learning_rate=0.1,
        objective='binary:logistic'
    )
)

Implementation

In [10]:
#Implementation

import requests
from datetime import datetime, timedelta
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, roc_auc_score
import warnings
warnings.filterwarnings('ignore')

# NASA POWER API Configuration
BASE_URL = "https://power.larc.nasa.gov/api/temporal/daily/point"

def fetch_nasa_power_data(lat, lon, start_date, end_date, parameters):
    """Fetch data from NASA POWER API"""
    params = {
        'parameters': ','.join(parameters),
        'community': 'ag',
        'longitude': lon,
        'latitude': lat,
        'start': start_date.strftime('%Y%m%d'),
        'end': end_date.strftime('%Y%m%d'),
        'format': 'json'
    }

    response = requests.get(BASE_URL, params=params)
    return response.json()

def process_nasa_data(json_data):
    """Convert NASA POWER JSON to DataFrame"""
    parameters = json_data['properties']['parameter']

    # Create DataFrame from nested dict
    df_dict = {}
    for param_name, param_data in parameters.items():
        df_dict[param_name] = param_data

    df = pd.DataFrame(df_dict)
    df.index = pd.to_datetime(df.index, format='%Y%m%d')
    df = df.replace(-999.0, np.nan)  # Handle fill values

    return df

def calculate_adverse_conditions(df, location_specific=True):
    """Create binary target variables for adverse conditions"""
    targets = pd.DataFrame(index=df.index)

    if location_specific:
        # Calculate percentile thresholds for each day of year
        for day in range(1, 367):
            mask = df.index.dayofyear == day
            if mask.sum() > 0:
                # Very Hot: 90th percentile
                threshold_hot = df.loc[mask, 'T2M_MAX'].quantile(0.90)
                targets.loc[mask, 'very_hot'] = (df.loc[mask, 'T2M_MAX'] > threshold_hot).astype(int)

                # Very Cold: 10th percentile
                threshold_cold = df.loc[mask, 'T2M_MIN'].quantile(0.10)
                targets.loc[mask, 'very_cold'] = (df.loc[mask, 'T2M_MIN'] < threshold_cold).astype(int)

                # Very Windy: 90th percentile
                if 'WS10M' in df.columns:
                    threshold_wind = df.loc[mask, 'WS10M'].quantile(0.90)
                    targets.loc[mask, 'very_windy'] = (df.loc[mask, 'WS10M'] > threshold_wind).astype(int)

                # Very Wet: 90th percentile of rainy days
                rainy_days = df.loc[mask & (df['PRECTOTCORR'] > 0), 'PRECTOTCORR']
                if len(rainy_days) > 0:
                    threshold_wet = rainy_days.quantile(0.90)
                    targets.loc[mask, 'very_wet'] = (df.loc[mask, 'PRECTOTCORR'] > threshold_wet).astype(int)

    # Absolute thresholds (safety-based)
    if 'WS10M' in df.columns:
        targets['very_windy_absolute'] = (df['WS10M'] >= 64.4).astype(int)  # 40 mph

    targets['very_wet_absolute'] = (df['PRECTOTCORR'] >= 76.2).astype(int)  # 3 inches

    # Calculate Heat Index (simplified)
    if 'RH2M' in df.columns:
        T = df['T2M_MAX'] * 9/5 + 32  # Convert to Fahrenheit
        R = df['RH2M']
        HI = -42.379 + 2.04901523*T + 10.14333127*R - 0.22475541*T*R
        targets['uncomfortable_hot'] = (HI > 103).astype(int)

    return targets.fillna(0)


def create_features(df):
    """Create predictive features for probability model"""
    features = pd.DataFrame(index=df.index)

    # Temporal features
    features['day_of_year'] = df.index.dayofyear
    features['month'] = df.index.month
    features['day_sin'] = np.sin(2 * np.pi * features['day_of_year'] / 365)
    features['day_cos'] = np.cos(2 * np.pi * features['day_of_year'] / 365)

    # Rolling statistics (7-day and 30-day)
    for window in [7, 30]:
        features[f'temp_mean_{window}d'] = df['T2M'].rolling(window).mean()
        features[f'temp_std_{window}d'] = df['T2M'].rolling(window).std()
        features[f'precip_sum_{window}d'] = df['PRECTOTCORR'].rolling(window).sum()
        features[f'precip_days_{window}d'] = (df['PRECTOTCORR'] > 0).rolling(window).sum()

    # Lag features
    for lag in [1, 7, 30]:
        features[f'temp_lag{lag}'] = df['T2M'].shift(lag)
        features[f'precip_lag{lag}'] = df['PRECTOTCORR'].shift(lag)

    # Climatological anomaly
    features['temp_anomaly'] = df['T2M'] - df.groupby(df.index.dayofyear)['T2M'].transform('mean')

    return features.dropna()

# Main execution
if __name__ == "__main__":
    # Configuration
    LAT, LON = 28.6, 77.2  # New Delhi
    START_DATE = datetime(1990, 1, 1)
    END_DATE = datetime(2023, 12, 31)

    PARAMETERS = ['T2M', 'T2M_MIN', 'T2M_MAX', 'PRECTOTCORR',
                  'WS10M', 'RH2M', 'PS']

    print("Fetching NASA POWER data...")
    raw_data = fetch_nasa_power_data(LAT, LON, START_DATE, END_DATE, PARAMETERS)

    print("Processing data...")
    df = process_nasa_data(raw_data)

    print("Creating target variables...")
    targets = calculate_adverse_conditions(df)

    print("Engineering features...")
    features_df = create_features(df)

    # Align features and targets
    common_idx = features_df.index.intersection(targets.index)
    X = features_df.loc[common_idx]
    y = targets.loc[common_idx]

    # Time-based split (important for time series)
    split_date = datetime(2020, 1, 1)
    X_train = X[X.index < split_date]
    X_test = X[X.index >= split_date]
    y_train = y[y.index < split_date]
    y_test = y[y.index >= split_date]

    print(f"\nTraining set: {len(X_train)} samples")
    print(f"Test set: {len(X_test)} samples")

    # Train model
    print("\nTraining multi-output classifier...")
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.multioutput import MultiOutputClassifier

    model = MultiOutputClassifier(
        RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    )

    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)

    # Evaluate
    print("\n" + "="*50)
    print("MODEL EVALUATION")
    print("="*50)

    for i, col in enumerate(y.columns):
        print(f"\n{col.upper()}:")
        # Check if there are any samples for this target in the test set
        if y_test.iloc[:, i].sum() == 0 and (1 - y_test.iloc[:, i]).sum() == 0:
            print("No samples for this target in the test set.")
            continue

        # Check unique classes in the test set for the current target
        unique_classes = y_test.iloc[:, i].unique()
        if len(unique_classes) == 1:
            # If only one class is present, adjust target_names and labels
            if unique_classes[0] == 0:
                print(classification_report(y_test.iloc[:, i], y_pred[:, i],
                                           target_names=['Normal'], labels=[0]))
            else:
                print(classification_report(y_test.iloc[:, i], y_pred[:, i],
                                           target_names=['Adverse'], labels=[1]))
        else:
            # If both classes are present, use default
            print(classification_report(y_test.iloc[:, i], y_pred[:, i],
                                       target_names=['Normal', 'Adverse']))


    # Save model
    import joblib
    joblib.dump(model, 'auracast_weather_model.pkl')
    print("\nModel saved as 'auracast_weather_model.pkl'")

Fetching NASA POWER data...
Processing data...
Creating target variables...
Engineering features...

Training set: 10927 samples
Test set: 1461 samples

Training multi-output classifier...

MODEL EVALUATION

VERY_HOT:
              precision    recall  f1-score   support

      Normal       0.97      0.98      0.97      1358
     Adverse       0.67      0.66      0.67       103

    accuracy                           0.95      1461
   macro avg       0.82      0.82      0.82      1461
weighted avg       0.95      0.95      0.95      1461


VERY_COLD:
              precision    recall  f1-score   support

      Normal       0.93      0.96      0.94      1265
     Adverse       0.64      0.51      0.57       196

    accuracy                           0.90      1461
   macro avg       0.78      0.73      0.75      1461
weighted avg       0.89      0.90      0.89      1461


VERY_WINDY:
              precision    recall  f1-score   support

      Normal       0.90      1.00      0.95     

Evaluation Metrics

In [7]:
#Evaluation Metrics

from sklearn.metrics import precision_recall_curve, average_precision_score

def evaluate_model(y_true, y_pred_proba, condition_names):
    """Comprehensive evaluation with business metrics"""
    results = {}

    for i, name in enumerate(condition_names):
        # Precision-Recall curve
        precision, recall, thresholds = precision_recall_curve(
            y_true[:, i], y_pred_proba[:, i]
        )

        # Average Precision Score
        ap_score = average_precision_score(y_true[:, i], y_pred_proba[:, i])

        # ROC-AUC
        roc_auc = roc_auc_score(y_true[:, i], y_pred_proba[:, i])

        results[name] = {
            'AP': ap_score,
            'ROC-AUC': roc_auc,
            'Precision': precision,
            'Recall': recall
        }

    return results

Core Implementation Strategy

In [11]:
import requests
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import brier_score_loss
import joblib

class AuraCastProbabilityModel:
    """
    Climatological probability model based on NASA POWER data
    Implements tercile-based probability forecasting with calibration
    """

    def __init__(self):
        self.model = None
        self.calibrator = None
        self.percentile_thresholds = {}

    def fetch_nasa_power_data(self, lat, lon, start_year, end_year):
        """Fetch multi-year data from NASA POWER API"""
        BASE_URL = "https://power.larc.nasa.gov/api/temporal/daily/point"

        start_date = f"{start_year}0101"
        end_date = f"{end_year}1231"

        params = {
            'parameters': 'T2M,T2M_MAX,T2M_MIN,PRECTOTCORR,WS10M,RH2M,PS',
            'community': 'ag',
            'longitude': lon,
            'latitude': lat,
            'start': start_date,
            'end': end_date,
            'format': 'json'
        }

        response = requests.get(BASE_URL, params=params)
        if response.status_code == 200:
            return response.json()
        else:
            raise Exception(f"API Error: {response.status_code}")

    def process_nasa_data(self, json_data):
        """Convert NASA JSON to DataFrame"""
        parameters = json_data['properties']['parameter']

        df_dict = {}
        for param_name, param_data in parameters.items():
            df_dict[param_name] = param_data

        df = pd.DataFrame(df_dict)
        df.index = pd.to_datetime(df.index, format='%Y%m%d')
        df = df.replace(-999.0, np.nan)

        return df

    def calculate_day_of_year_statistics(self, df, window=15):
        """
        Calculate percentile thresholds for each day of year
        Uses ±window days for robust statistics
        """
        stats = {}

        for target_day in range(1, 367):
            # Create mask for days within window
            day_mask = (
                (df.index.dayofyear >= target_day - window) &
                (df.index.dayofyear <= target_day + window)
            )
            window_data = df[day_mask]

            if len(window_data) < 10:  # Minimum sample size
                continue

            stats[target_day] = {
                'temp_max_90p': window_data['T2M_MAX'].quantile(0.90),
                'temp_max_10p': window_data['T2M_MAX'].quantile(0.10),
                'temp_min_90p': window_data['T2M_MIN'].quantile(0.90),
                'temp_min_10p': window_data['T2M_MIN'].quantile(0.10),
                'precip_90p': window_data[window_data['PRECTOTCORR'] > 0]['PRECTOTCORR'].quantile(0.90),
                'wind_90p': window_data['WS10M'].quantile(0.90) if 'WS10M' in window_data else None,
                'sample_size': len(window_data)
            }

        return stats

    def create_tercile_targets(self, df, stats):
        """
        Create tercile-based probability targets
        Based on WCS methodology from your documents
        """
        targets = pd.DataFrame(index=df.index)

        for idx, row in df.iterrows():
            day = idx.dayofyear
            if day not in stats:
                continue

            # Very Hot: Above 90th percentile (upper tercile)
            targets.loc[idx, 'very_hot'] = int(row['T2M_MAX'] > stats[day]['temp_max_90p'])

            # Very Cold: Below 10th percentile (lower tercile)
            targets.loc[idx, 'very_cold'] = int(row['T2M_MIN'] < stats[day]['temp_min_10p'])

            # Very Wet: Above 90th percentile of rainy days
            if pd.notna(row['PRECTOTCORR']) and row['PRECTOTCORR'] > 0:
                targets.loc[idx, 'very_wet'] = int(row['PRECTOTCORR'] > stats[day]['precip_90p'])
            else:
                targets.loc[idx, 'very_wet'] = 0

            # Very Windy: Above 90th percentile OR absolute threshold
            if 'WS10M' in row and pd.notna(row['WS10M']):
                targets.loc[idx, 'very_windy'] = int(
                    (row['WS10M'] > stats[day]['wind_90p']) or (row['WS10M'] >= 17.9)  # 40mph
                )

        return targets.dropna()

    def engineer_features(self, df):
        """Create predictive features for probability model"""
        features = pd.DataFrame(index=df.index)

        # Temporal features
        features['day_of_year'] = df.index.dayofyear
        features['month'] = df.index.month
        features['day_sin'] = np.sin(2 * np.pi * features['day_of_year'] / 365)
        features['day_cos'] = np.cos(2 * np.pi * features['day_of_year'] / 365)

        # Rolling statistics (7-day and 30-day)
        for window in [7, 30]:
            features[f'temp_mean_{window}d'] = df['T2M'].rolling(window).mean()
            features[f'temp_std_{window}d'] = df['T2M'].rolling(window).std()
            features[f'precip_sum_{window}d'] = df['PRECTOTCORR'].rolling(window).sum()
            features[f'precip_days_{window}d'] = (df['PRECTOTCORR'] > 0).rolling(window).sum()

        # Lag features
        for lag in [1, 7, 30]:
            features[f'temp_lag{lag}'] = df['T2M'].shift(lag)
            features[f'precip_lag{lag}'] = df['PRECTOTCORR'].shift(lag)

        # Climatological anomaly
        features['temp_anomaly'] = df['T2M'] - df.groupby(df.index.dayofyear)['T2M'].transform('mean')

        return features.dropna()

    def train_calibrated_model(self, X_train, y_train):
        """
        Train probability model with calibration
        Implements reliability framework from your documents
        """
        # Base classifier
        base_model = RandomForestClassifier(
            n_estimators=200,
            max_depth=15,
            min_samples_split=50,
            min_samples_leaf=20,
            random_state=42,
            n_jobs=-1
        )

        # Calibration for reliable probabilities (isotonic regression)
        self.model = CalibratedClassifierCV(
            base_model,
            method='isotonic',
            cv=5
        )

        self.model.fit(X_train, y_train)
        return self

    def calculate_reliability_metrics(self, y_true, y_pred_proba, n_bins=10):
        """
        Calculate reliability diagram data
        Based on WCS reliability assessment methodology
        """
        bin_edges = np.linspace(0, 1, n_bins + 1)
        bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

        observed_freq = []
        forecast_prob = []
        bin_counts = []

        for i in range(n_bins):
            mask = (y_pred_proba >= bin_edges[i]) & (y_pred_proba < bin_edges[i+1])
            if mask.sum() > 0:
                observed_freq.append(y_true[mask].mean())
                forecast_prob.append(bin_centers[i])
                bin_counts.append(mask.sum())

        return {
            'forecast_prob': forecast_prob,
            'observed_freq': observed_freq,
            'counts': bin_counts,
            'brier_score': brier_score_loss(y_true, y_pred_proba)
        }

    def predict_probabilities(self, X, target_date):
        """
        Predict tercile probabilities for target date
        Returns probabilities aligned with your frontend structure
        """
        if self.model is None:
            raise Exception("Model not trained. Call train_calibrated_model first.")

        # Get probabilities for each adverse condition
        probabilities = {}

        for condition in ['very_hot', 'very_cold', 'very_windy', 'very_wet']:
            prob = self.model.predict_proba(X)[:, 1]  # Probability of adverse condition
            probabilities[condition] = {
                'probability': float(prob[0]),
                'threshold': self.get_threshold_for_date(target_date, condition),
                'confidence': self.calculate_confidence_score(X)
            }

        return probabilities

    def calculate_confidence_score(self, X):
        """
        Calculate statistical confidence based on data quality
        Your research mentions this as key trust metric
        """
        # Factors: sample size, data completeness, model uncertainty
        base_confidence = 0.85

        # Adjust for feature completeness
        missing_ratio = X.isna().sum().sum() / X.size
        completeness_penalty = missing_ratio * 0.2

        return max(0.6, base_confidence - completeness_penalty)

# Main execution script
def main():
    print("="*60)
    print("AURACAST PROBABILITY MODEL TRAINING")
    print("="*60)

    # Configuration
    LAT, LON = 28.6, 77.2  # New Delhi
    START_YEAR = 1990
    END_YEAR = 2023

    # Initialize model
    model = AuraCastProbabilityModel()

    # Fetch data
    print(f"\n1. Fetching NASA POWER data ({START_YEAR}-{END_YEAR})...")
    raw_data = model.fetch_nasa_power_data(LAT, LON, START_YEAR, END_YEAR)

    # Process data
    print("2. Processing weather data...")
    df = model.process_nasa_data(raw_data)
    print(f"   Data points: {len(df)}")

    # Calculate climatological statistics
    print("3. Calculating day-of-year statistics...")
    stats = model.calculate_day_of_year_statistics(df, window=15)
    model.percentile_thresholds = stats

    # Create targets
    print("4. Creating tercile-based targets...")
    targets = model.create_tercile_targets(df, stats)

    # Engineer features
    print("5. Engineering predictive features...")
    features = model.engineer_features(df)

    # Align data
    common_idx = features.index.intersection(targets.index)
    X = features.loc[common_idx]
    y = targets.loc[common_idx]

    # Time-based split
    split_date = datetime(2020, 1, 1)
    train_mask = X.index < split_date
    test_mask = X.index >= split_date

    X_train, X_test = X[train_mask], X[test_mask]
    y_train, y_test = y[train_mask], y[test_mask]

    print(f"\n   Training samples: {len(X_train)}")
    print(f"   Test samples: {len(X_test)}")

    # Train models for each condition
    print("\n6. Training calibrated probability models...")
    results = {}

    for condition in ['very_hot', 'very_cold', 'very_windy', 'very_wet']:
        if condition not in y_train.columns:
            continue

        print(f"\n   Training: {condition}")
        model_cond = AuraCastProbabilityModel()
        model_cond.train_calibrated_model(X_train, y_train[condition])

        # Predict probabilities
        y_pred_proba = model_cond.model.predict_proba(X_test)[:, 1]

        # Calculate reliability
        reliability = model_cond.calculate_reliability_metrics(
            y_test[condition].values,
            y_pred_proba
        )

        results[condition] = {
            'model': model_cond,
            'brier_score': reliability['brier_score'],
            'reliability_data': reliability
        }

        print(f"      Brier Score: {reliability['brier_score']:.4f}")
        print(f"      (Lower is better, perfect = 0.0)")

    # Save models
    print("\n7. Saving models...")
    joblib.dump(results, 'auracast_probability_models.pkl')
    joblib.dump(stats, 'auracast_climatology_stats.pkl')

    print("\n" + "="*60)
    print("MODEL TRAINING COMPLETE")
    print("="*60)
    print(f"\nModels saved:")
    print("  - auracast_probability_models.pkl")
    print("  - auracast_climatology_stats.pkl")

    # Example prediction
    print("\n" + "="*60)
    print("EXAMPLE PREDICTION")
    print("="*60)
    target_date = datetime(2024, 7, 15)
    print(f"\nQuery: Risk assessment for {target_date.strftime('%B %d, %Y')}")

    # Get last 30 days of features
    sample_features = X_test.iloc[-1:].copy()

    for condition, result in results.items():
        prob = result['model'].model.predict_proba(sample_features)[:, 1][0]
        print(f"\n{condition.replace('_', ' ').title()}:")
        print(f"  Probability: {prob*100:.1f}%")
        print(f"  Model Quality (Brier): {result['brier_score']:.4f}")

if __name__ == "__main__":
    main()

AURACAST PROBABILITY MODEL TRAINING

1. Fetching NASA POWER data (1990-2023)...
2. Processing weather data...
   Data points: 12418
3. Calculating day-of-year statistics...
4. Creating tercile-based targets...
5. Engineering predictive features...

   Training samples: 10927
   Test samples: 1461

6. Training calibrated probability models...

   Training: very_hot
      Brier Score: 0.0244
      (Lower is better, perfect = 0.0)

   Training: very_cold
      Brier Score: 0.0604
      (Lower is better, perfect = 0.0)

   Training: very_windy
      Brier Score: 0.0798
      (Lower is better, perfect = 0.0)

   Training: very_wet
      Brier Score: 0.0404
      (Lower is better, perfect = 0.0)

7. Saving models...

MODEL TRAINING COMPLETE

Models saved:
  - auracast_probability_models.pkl
  - auracast_climatology_stats.pkl

EXAMPLE PREDICTION

Query: Risk assessment for July 15, 2024

Very Hot:
  Probability: 0.0%
  Model Quality (Brier): 0.0244

Very Cold:
  Probability: 0.4%
  Model Qual